In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score, average_precision_score, make_scorer
import optuna

import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['TF_USE_LEGACY_KERAS'] = '1' 

from functions import *
#from bert_vectorizer import BertVectorizer

# Load your dataset
df = pd.read_csv("../data/clean/dataset.csv").fillna('')
X = df["title"]
y = df["funny"]

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

2024-11-24 23:10:33.851352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732486234.158853    3692 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732486234.248492    3692 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 23:10:34.949372: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import tensorflow as tf
#from tensorflow.keras import mixed_precision

# Enable mixed precision training
#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_global_policy(policy)
#tf.keras.mixed_precision.set_global_policy('float16')

# Load tokenizer and model
model_name = "camembert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Change `num_labels` for your task

# Prepare dataset
def preprocess_function(sentences, labels, tokenizer, max_length=128):
    inputs = tokenizer(sentences, truncation=True, padding=True, max_length=max_length, return_tensors="tf")
    dataset = tf.data.Dataset.from_tensor_slices((dict(inputs), labels))
    return dataset.batch(8)

train_dataset = preprocess_function(list(X_train), list(y_train), tokenizer)

# Compile and train the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

model.fit(train_dataset, epochs=3)

I0000 00:00:1732486241.739826    3692 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1732486241.742269    3692 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1730 MB memory:  -> device: 0, name: NVIDIA GeForce MX150, pci bus id: 0000:01:00.0, compute capability: 6.1
All PyTorch model weights were used when initializing TFCamembertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFCamembertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


I0000 00:00:1732486276.089839    3752 service.cc:148] XLA service 0x7fc3a09a47a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732486276.090308    3752 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce MX150, Compute Capability 6.1
2024-11-24 23:11:16.130606: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732486276.212214    3752 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-11-24 23:11:16.267197: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.4 which is older than the PTX compiler version 12.5.82. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
I0000 00:00:173248627

10657/10657 [==============================] - 5981s 558ms/step - loss: 0.1340 - accuracy: 0.9493
Epoch 2/3
10657/10657 [==============================] - 5921s 556ms/step - loss: 0.0695 - accuracy: 0.9750
Epoch 3/3
10657/10657 [==============================] - 5919s 555ms/step - loss: 0.0396 - accuracy: 0.9865


In [3]:
# Save model and tokenizer
save_path = "../model_exports/fine_tuned_bert"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('../model_exports/fine_tuned_bert/tokenizer_config.json',
 '../model_exports/fine_tuned_bert/special_tokens_map.json',
 '../model_exports/fine_tuned_bert/sentencepiece.bpe.model',
 '../model_exports/fine_tuned_bert/added_tokens.json',
 '../model_exports/fine_tuned_bert/tokenizer.json')

In [4]:
# To Reload the model later

#from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

# Load model and tokenizer
#model = TFAutoModelForSequenceClassification.from_pretrained(save_path)
#tokenizer = AutoTokenizer.from_pretrained(save_path)


In [5]:
# Preprocess the test data
test_dataset = preprocess_function(list(X_test), list(y_test), tokenizer)


In [6]:
# Evaluate the model on the test dataset
results = model.evaluate(test_dataset)

# Print results
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")


2665/2665 [==============================] - 533s 199ms/step - loss: 0.1505 - accuracy: 0.9605
Test Loss: 0.15052025020122528
Test Accuracy: 0.9604936242103577


In [7]:
# Generate predictions
predictions = model.predict(test_dataset)
predicted_labels = tf.argmax(predictions.logits, axis=1).numpy()

# Evaluate with additional metrics
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted_labels, target_names=["Class 0", "Class 1"]))

2665/2665 [==============================] - 523s 195ms/step
              precision    recall  f1-score   support

     Class 0       0.96      0.99      0.98     17359
     Class 1       0.94      0.84      0.89      3954

    accuracy                           0.96     21313
   macro avg       0.95      0.91      0.93     21313
weighted avg       0.96      0.96      0.96     21313



In [8]:
from scipy.special import softmax
# Convert logits to probabilities using softmax
probas = softmax(predictions.logits, axis=1)

In [11]:
results_df = pd.DataFrame({
    "y_true": y_test,
    "y_pred": predicted_labels,
    "proba": probas[:,1],
    "title": X_test, 
})

false_positives = results_df[(results_df["y_true"] == 0) & (results_df["y_pred"] == 1)]

true_positives = results_df[(results_df["y_true"] == 1) & (results_df["y_pred"] == 1)]

sorted_false_positives = false_positives.sort_values(by="proba", ascending=False)

sorted_true_positives = true_positives.sort_values(by="proba", ascending=False)


print(sorted_false_positives.head())
print(sorted_true_positives.head())

       y_true  y_pred     proba  \
95199       0       1  0.998561   
23559       0       1  0.998519   
53181       0       1  0.998206   
94885       0       1  0.998069   
92461       0       1  0.997829   

                                                   title  
95199  VIDÉO. L'homme qui murmure à l'oreille des c...  
23559  VIDEO. Une "trombe marine" se transforme en to...  
53181  Il est désormais possible de parier sur le rés...  
94885  Ces ralentisseurs sont illégaux, on vous dit ...  
92461  Elle disait vouloir « manger des enfants et le...  
       y_true  y_pred     proba  \
28206       1       1  0.998979   
33876       1       1  0.998978   
28383       1       1  0.998978   
50252       1       1  0.998977   
28953       1       1  0.998977   

                                                   title  
28206  457 guitaristes jouent "Highway to Hell" d'AC/...  
33876  VIDÉO. Les fenêtres à bascule de ce gratte-...  
28383  Ce raton laveur (ou plutôt escaladeur) 

In [12]:
for index, row in sorted_false_positives.head(100).iterrows():
    print(row['title'], row['proba'])

VIDÉO. L'homme qui murmure à l'oreille des cachalots 0.998561441898346
VIDEO. Une "trombe marine" se transforme en tornade et ravage des champs de tournesols 0.9985188841819763
Il est désormais possible de parier sur le résultat de l'élection présidentielle américaine 0.9982055425643921
Ces ralentisseurs sont illégaux, on vous dit pourquoi 0.9980688691139221
Elle disait vouloir « manger des enfants et les dessiner nus » sur le tchat en ligne de la police 0.9978287816047668
Des parfums qui ont du chien 0.9975473284721375
Joe Biden: un honnête homme s'en va 0.9973794221878052
Aux États-Unis, des policiers interviennent pour dégager une citrouille géante gonflable de la route 0.996993899345398
Le premier qui dit la vérité se fait étriller 0.9966431856155396
Un cheval remporte une course avec une cote colossale... mais meurt juste après 0.9962510466575623
Royaume-Uni : pour son anniversaire, une centenaire s'offre un saut en parachute 0.996246874332428
L'Euro de foot des sans-abri 0

In [13]:
for index, row in sorted_true_positives.head(100).iterrows():
    print(row['title'], row['proba'])

457 guitaristes jouent "Highway to Hell" d'AC/DC en même temps et ça fait beaucoup de bruit 0.998979389667511
VIDÉO. Les fenêtres à bascule de ce gratte-ciel de Chicago vont vous donner le vertige 0.9989783763885498
Ce raton laveur (ou plutôt escaladeur) n'a rien à envier à Spider-Man 0.9989781379699707
La réaction de ce couple d'agriculteurs en apprenant qu'il a gagné au Loto va vous émouvoir 0.9989774227142334
Ces Coréens rois du beatbox reprennent "Despacito" et c'est parfait 0.9989768266677856
VIDÉO. La police néo-zélandaise recrute et elle l'a fait savoir d'une drôle de manière 0.9989707469940186
Ce pilote d'avion est d'une franchise confondante 0.9989686012268066
En mangeant 75 hot-dogs d'affilé, cet Américain a battu son propre record du monde 0.9989684820175171
VIDEO. Ce petit garçon qui tente de casser une planche au Taekwondo va vous faire fondre 0.9989681243896484
VIDÉO. Un homme essaie de sabrer en deux une pastèque et c'est une mauvaise idée 0.998967647